## Regression Analysis Examples

using data from https://vincentarelbundock.github.io/Rdatasets/articles/data.html, Determinants of Wages Data (CPS 1988).

In [24]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

### Data Pre-processing

In [4]:
df = pd.read_csv('CPS1988.csv')
df = df.drop(['Unnamed: 0'], axis=1)

cat_cols = ['ethnicity','smsa','region','parttime']
df[cat_cols] =df[cat_cols].astype('category')
df[cat_cols] = df[cat_cols].apply(lambda x: x.cat.codes)
print(df.dtypes)
df.head()

wage          float64
education       int64
experience      int64
ethnicity        int8
smsa             int8
region           int8
parttime         int8
dtype: object


,wage,education,experience,ethnicity,smsa,region,parttime
0,354.94,7,45,1,1,1,0
1,123.46,12,1,1,1,1,1
2,370.37,9,9,1,1,1,0
3,754.94,11,46,1,1,1,0
4,593.54,12,36,1,1,1,0


One-hot encode and display for visual check

In [5]:
df = pd.get_dummies(df, columns=['region'])
df.head()

,wage,education,experience,ethnicity,smsa,parttime,region_0,region_1,region_2,region_3
0,354.94,7,45,1,1,0,0,1,0,0
1,123.46,12,1,1,1,1,0,1,0,0
2,370.37,9,9,1,1,0,0,1,0,0
3,754.94,11,46,1,1,0,0,1,0,0
4,593.54,12,36,1,1,0,0,1,0,0


#### Train/Test Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('wage',axis=1),df.wage)

#### Linear Regression

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)
print( "MSE Train: {}".format(mean_squared_error(y_train, lr.predict(X_train))))
print( "MSE Test: {}".format(mean_squared_error(y_test, lr.predict(X_test))))

print( "r2 Train: {}".format(r2_score(y_train, lr.predict(X_train))))
print( "r2 Test: {}".format(r2_score(y_test, lr.predict(X_test))))

MSE Train: 155137.8448844423
MSE Test: 157441.77771282007
r2 Train: 0.24855475114140502
r2 Test: 0.2252785728368033


Tune Linear Regression

In [19]:
lr_pipe = make_pipeline(PolynomialFeatures(2), StandardScaler(), LinearRegression())
lr_pipe.fit(X_train, y_train)

print( "MSE Train: {}".format(mean_squared_error(y_train, lr_pipe.predict(X_train))))
print( "MSE Test: {}".format(mean_squared_error(y_test, lr_pipe.predict(X_test))))

print( "r2 Train: {}".format(r2_score(y_train, lr_pipe.predict(X_train))))
print( "r2 Test: {}".format(r2_score(y_test, lr_pipe.predict(X_test))))

MSE Train: 146451.94364748526
MSE Test: 147585.03879117838
r2 Train: 0.290626878812304
r2 Test: 0.27378048227584717


In [25]:
print(lr_pipe.get_params().keys())
param_grid = [{'polynomialfeatures__interaction_only': [True,False], 
               'polynomialfeatures__degree': [1,2,3]}]

grid_search = GridSearchCV(lr_pipe,param_grid, cv=5, scoring='neg_mean_squared_error',return_train_score=True,verbose=0)
grid_search.fit(X_train, y_train)

dict_keys(['memory', 'steps', 'verbose', 'polynomialfeatures', 'standardscaler', 'linearregression', 'polynomialfeatures__degree', 'polynomialfeatures__include_bias', 'polynomialfeatures__interaction_only', 'polynomialfeatures__order', 'standardscaler__copy', 'standardscaler__with_mean', 'standardscaler__with_std', 'linearregression__copy_X', 'linearregression__fit_intercept', 'linearregression__n_jobs', 'linearregression__normalize', 'linearregression__positive'])


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('standardscaler', StandardScaler()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid=[{'polynomialfeatures__degree': [1, 2, 3],
                          'polynomialfeatures__interaction_only': [True,
                                                                   False]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [26]:
best_lr=grid_search.best_estimator_
print(grid_search.best_params_)
print( "MSE Train: {}".format(mean_squared_error(y_train, best_lr.predict(X_train))))
print( "MSE Test: {}".format(mean_squared_error(y_test, best_lr.predict(X_test))))

print( "r2 Train: {}".format(r2_score(y_train, best_lr.predict(X_train))))
print( "r2 Test: {}".format(r2_score(y_test, best_lr.predict(X_test))))

{'polynomialfeatures__degree': 2, 'polynomialfeatures__interaction_only': False}
MSE Train: 146451.94364748526
MSE Test: 147585.03879117838
r2 Train: 0.290626878812304
r2 Test: 0.27378048227584717


### Lasso

In [28]:
lasso_pipe = make_pipeline(PolynomialFeatures(2), StandardScaler(), Lasso())
lasso_pipe.fit(X_train, y_train)
print( "MSE Train: {}".format(mean_squared_error(y_train, lasso_pipe.predict(X_train))))
print( "MSE Test: {}".format(mean_squared_error(y_test, lasso_pipe.predict(X_test))))

print( "r2 Train: {}".format(r2_score(y_train, lasso_pipe.predict(X_train))))
print( "r2 Test: {}".format(r2_score(y_test, lasso_pipe.predict(X_test))))

MSE Train: 146995.83998569564
MSE Test: 148295.27470198614
r2 Train: 0.28799239385137
r2 Test: 0.2702856349333127


### Ridge

In [29]:
ridge_pipe = make_pipeline(PolynomialFeatures(2), StandardScaler(), Ridge())
ridge_pipe.fit(X_train, y_train)
print( "MSE Train: {}".format(mean_squared_error(y_train, ridge_pipe.predict(X_train))))
print( "MSE Test: {}".format(mean_squared_error(y_test, ridge_pipe.predict(X_test))))

print( "r2 Train: {}".format(r2_score(y_train, ridge_pipe.predict(X_train))))
print( "r2 Test: {}".format(r2_score(y_test, ridge_pipe.predict(X_test))))


MSE Train: 146451.0848457076
MSE Test: 147585.85701403703
r2 Train: 0.29063103861300255
r2 Test: 0.27377645605872736


### Lasso with Grid Search

In [32]:
lasso_pipe = make_pipeline(PolynomialFeatures(2), StandardScaler(), Lasso())
# print(lasso_pipe.get_params().keys())
param_grid = [{'polynomialfeatures__interaction_only': [True,False], 
               'polynomialfeatures__degree': [1,2,3],
               'lasso__alpha': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
              }]

grid_search = GridSearchCV(lasso_pipe,param_grid, cv=5, scoring='neg_mean_squared_error',return_train_score=True,verbose=0)
grid_search.fit(X_train, y_train)

C:\Users\patri\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 342866299.32220054, tolerance: 361299.6285164941
  model = cd_fast.enet_coordinate_descent(
C:\Users\patri\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 347258647.1161797, tolerance: 362878.7618735275
  model = cd_fast.enet_coordinate_descent(
C:\Users\patri\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 317944081.14920306, tolerance: 337078.58202624036
  model = cd_fast.enet_coordinate_descent(
C:\Users\patri\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWar

C:\Users\patri\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 48065089.28788495, tolerance: 361672.49557527143
  model = cd_fast.enet_coordinate_descent(
C:\Users\patri\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22810008.66395259, tolerance: 320824.36249666853
  model = cd_fast.enet_coordinate_descent(
C:\Users\patri\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1155702.9699363708, tolerance: 362878.7618735275
  model = cd_fast.enet_coordinate_descent(
C:\Users\patri\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWar

C:\Users\patri\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 611467.7329645157, tolerance: 320824.36249666853
  model = cd_fast.enet_coordinate_descent(
C:\Users\patri\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 709817.1972723007, tolerance: 337078.58202624036
  model = cd_fast.enet_coordinate_descent(
C:\Users\patri\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2651530.891442299, tolerance: 361672.49557527143
  model = cd_fast.enet_coordinate_descent(
C:\Users\patri\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWar

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('standardscaler', StandardScaler()),
                                       ('lasso', Lasso())]),
             param_grid=[{'lasso__alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,
                                           0.8, 0.9, 1.0],
                          'polynomialfeatures__degree': [1, 2, 3],
                          'polynomialfeatures__interaction_only': [True,
                                                                   False]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [33]:
best_lr=grid_search.best_estimator_
print(grid_search.best_params_)
print( "MSE Train: {}".format(mean_squared_error(y_train, best_lr.predict(X_train))))
print( "MSE Test: {}".format(mean_squared_error(y_test, best_lr.predict(X_test))))

print( "r2 Train: {}".format(r2_score(y_train, best_lr.predict(X_train))))
print( "r2 Test: {}".format(r2_score(y_test, best_lr.predict(X_test))))

{'lasso__alpha': 0.2, 'polynomialfeatures__degree': 3, 'polynomialfeatures__interaction_only': False}
MSE Train: 145248.910180241
MSE Test: 146982.123190095
r2 Train: 0.29645404357569205
r2 Test: 0.27674724015749574


### Ridge with Grid Search

In [34]:
ridge_pipe = make_pipeline(PolynomialFeatures(2), StandardScaler(), Ridge())
# print(ridge_pipe.get_params().keys())
param_grid = [{'polynomialfeatures__interaction_only': [True,False], 
               'polynomialfeatures__degree': [1,2,3],
               'ridge__alpha': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
              }]

grid_search = GridSearchCV(ridge_pipe,param_grid, cv=5, scoring='neg_mean_squared_error',return_train_score=True,verbose=0)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('standardscaler', StandardScaler()),
                                       ('ridge', Ridge())]),
             param_grid=[{'polynomialfeatures__degree': [1, 2, 3],
                          'polynomialfeatures__interaction_only': [True, False],
                          'ridge__alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,
                                           0.8, 0.9, 1.0]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [35]:
best_lr=grid_search.best_estimator_
print(grid_search.best_params_)
print( "MSE Train: {}".format(mean_squared_error(y_train, best_lr.predict(X_train))))
print( "MSE Test: {}".format(mean_squared_error(y_test, best_lr.predict(X_test))))

print( "r2 Train: {}".format(r2_score(y_train, best_lr.predict(X_train))))
print( "r2 Test: {}".format(r2_score(y_test, best_lr.predict(X_test))))

{'polynomialfeatures__degree': 3, 'polynomialfeatures__interaction_only': False, 'ridge__alpha': 1.0}
MSE Train: 144823.36383906822
MSE Test: 147342.25483304125
r2 Train: 0.29851527355140584
r2 Test: 0.274975145708087


In [36]:
class CustomGDRegressor(object):
    def __init__(self, input_size, output_size=1):
        self.input_size = input_size+1
        self.output_size = output_size
        self.B =  np.random.randn( self.output_size, self.input_size,)
        self.cost = []
        
    def errors(self, y_pred, y):
        return y-y_pred
    
    def predict(self, X):
        return  np.dot(self.B, X.T).T
    
    def cost_function(self, errors):
        return (1/(2*len(errors))) * sum(errors*errors)
    
    def fit_by_input(self, _X, y):
        x_len = len(_X)
        y_len = len(y)
        y = y.reshape(y_len, 1)
        X = np.c_[np.ones(x_len),_X]
        
        if x_len != y_len:
            raise("X and y not same length")
            
        iter_count = 1000
        lr = 0.01
        
        for _ in range(iter_count):
            w_ = np.zeros((1, self.input_size))
            for x_, y_ in zip(X, y):
                
                y_p = np.dot(x_, self.B.T)
                err = y_-y_p
                
                w_ += err * x_
                
            self.B +=w_ * lr *(1/x_len)
            
            self.cost.append(sum(self.errors(self.predict(X), y)**2))
        return self.B
        
    def fit(self, _X, y):
        
        def learning_rate(lr, decay=1.1):
            return max(0.000001, lr/decay)
        
        x_len = len(_X)
        y_len = len(y)
        y = y.reshape(y_len, 1)
        X = np.c_[np.ones(x_len),_X]
        
        if x_len != y_len:
            raise("X and y not same length")
            
        iter_count = 1000
        lr = 0.01
        for _ in range(iter_count):
            y_p = self.predict(X)
            
            err = self.errors(y_p, y)
            self.B[0] = self.B[0] + lr * (1/x_len)*sum(err)
            
            err_ = err*X
        
            self.B = self.B + lr * sum(err_)*(1/x_len)
            self.cost.append(sum(self.errors(self.predict(X), y)**2))
            lr = learning_rate(lr,decay=1)
        return self.B
    
    def fit_matrix_method(self, _X, y):
        """ (X'X)^-1 X'Y = b """
        X = np.c_[np.ones(len(_X)),_X]
        self.B = matmul(matmul(inv( matmul(X.T,X)),X.T),y)
        self.cost.append(sum(self.errors(self.predict(X), y)**2))
        return self.B
        
        